In [1]:
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables from the .env file using 'from dotenv import find_dotenv, load_dotenv'
load_dotenv(find_dotenv(filename='SURF-Project_Optimizing-PerunaBot/setup/.env'))

True

In [2]:
from OG_PerunaBot_chain import Original_PerunaBot_eval_chain
from chain_0 import base_retriever_eval_chain_0
from chain_1 import parent_retriever_eval_chain_1
from chain_2 import ensemble_retriever_eval_chain_2

In [3]:
#langsmith
from langsmith import Client
import os

langsmith_api_key = os.environ["LANGSMITH_API_KEY"]
langchain_endpoint = os.environ["LANGCHAIN_ENDPOINT"]
langsmith_project = os.environ["LANGCHAIN_PROJECT"]
os.environ["LANGCHAIN_TRACING_V2"]

# Initialize LangSmith Client using 'from langsmith import Client'
langsmith_client = Client()

In [ ]:
from OG_PerunaBot_chain import Original_PerunaBot_eval_chain
from chain_0 import base_retriever_eval_chain_0
from chain_1 import parent_retriever_eval_chain_1
from chain_2 import ensemble_retriever_eval_chain_2

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [21]:

def construct_OG_chain():
    def predict(inputs: dict) -> dict:
        text = inputs.get("question")
        input_mapper = {'text': 'question', 'input': text}
        my_chain = input_mapper | Original_PerunaBot_eval_chain
        response = my_chain.invoke({"question": input_mapper['input']})
        return response["answer"]
    


In [22]:
from langchain.smith import RunEvalConfig, run_on_dataset
from langchain.evaluation import EvaluatorType, load_evaluator
from langchain.evaluation.criteria import CriteriaEvalChain, Criteria
from langchain_openai import ChatOpenAI


# datasets in langsmith
data_set_1 = "SMU Schools Basic Info"
data_set_2 = "SMU Campus Facts"
project_name = "First test eval for "

eval_llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Evaluation configuration
eval_config = RunEvalConfig(
    evaluators = [
        "qa",
        "cot_qa",
        "context_qa"
    ],
    eval_llm = ChatOpenAI(model="gpt-4o", temperature=0)
)



# Evaluate the target task
def evaluate_chain(chain, dataset, chain_name):
    chain_results = run_on_dataset(
        client=langsmith_client,
        llm_or_chain_factory=construct_chain,
        evaluation=eval_config,
        dataset_name=dataset,
        verbose=True,
        project_name= project_name + chain_name,
        project_metadata={
            "chain": chain_name,
            "dataset": dataset,
            "version": "0.1"
        }
    )

In [23]:

evaluate_chain(Original_PerunaBot_eval_chain, data_set_1, "Original PerunaBot chain")

View the evaluation results for project 'First test eval for Original PerunaBot chain' at:
https://smith.langchain.com/o/6f55b15c-54b8-5d1e-b020-8c8816d39599/datasets/d9b7f7ca-c259-4b1b-b464-017e5fc9395a/compare?selectedSessions=dfb2c94f-d3b3-4718-926b-2a5bdb38f099

View all tests for Dataset SMU Schools Basic Info at:
https://smith.langchain.com/o/6f55b15c-54b8-5d1e-b020-8c8816d39599/datasets/d9b7f7ca-c259-4b1b-b464-017e5fc9395a
[------------------------------------------------->] 8/8

,feedback.correctness,feedback.COT Contextual Accuracy,feedback.Contextual Accuracy,error,execution_time,run_id
count,8.0,8.0,8.0,0,8.000000,8
unique,NaN,NaN,NaN,0,NaN,8
top,NaN,NaN,NaN,NaN,NaN,4d84bb35-81ec-411a-8a0d-00bcc5882860
freq,NaN,NaN,NaN,NaN,NaN,1
mean,0.0,0.0,0.0,NaN,0.012205,NaN
std,0.0,0.0,0.0,NaN,0.005659,NaN
min,0.0,0.0,0.0,NaN,0.004012,NaN
25%,0.0,0.0,0.0,NaN,0.009935,NaN
50%,0.0,0.0,0.0,NaN,0.011417,NaN
75%,0.0,0.0,0.0,NaN,0.013905,NaN


In [ ]:

# Chain 0 on data set 1
evaluate_chain(base_retriever_eval_chain_0, data_set_1, "Base Retriever Chain")

In [ ]:

# Chain 1 on data set 1
evaluate_chain(parent_retriever_eval_chain_1, data_set_1, "Parent Retriever Chain")

In [ ]:

# Chain 2 on data set 1
evaluate_chain(ensemble_retriever_eval_chain_2, data_set_1, "Ensemble Retriever Chain")

In [ ]:
# OG PerunaBot Chain on data set 2
evaluate_chain(Original_PerunaBot__eval_chain, data_set_2, "Original PerunaBot chain")

In [ ]:

# Chain 0 on data set 2
evaluate_chain(base_retriever__eval_chain_0, data_set_2, "Base Retriever Chain")

In [ ]:

# Chain 1 on data set 2
evaluate_chain(parent_retriever__eval_chain_1, data_set_2, "Parent Retriever Chain")

In [ ]:

# Chain 2 on data set 2
evaluate_chain(ensemble_retriever__eval_chain_2, data_set_2, "Ensemble Retriever Chain")